In [10]:
#Import modules
import requests
from config import news_key, int_key
from newsapi import NewsApiClient
import json
import pandas as pd
from datetime import datetime, timedelta


In [11]:
#Get only weekdays in an array so we can search the News API
Tickers= ["AAPL", "BRK.A", "JPM", "XOM", "T", "BAC", "WFC", "VZ", "MSFT", "WMT", "GOOGL", "AMZN"]
stock = requests.get(f"https://api.iextrading.com/1.0/stock/{Tickers[0]}/chart/1m")
stocks = stock.json()
dates = []

for i in range(len(stocks)):
    dates.append(stocks[i]["date"])
#take off the first date if we are outside of the news API date frame
dates.remove(dates[0])


In [12]:
companies= ["Apple", "Berkshire Hathaway", "JP Morgan Chase", "ExxonMobil", "AT&T", "Bank of America", "Wells Fargo", 
            "Verizon", "Microsoft", "Wal-Mart", "Alphabet", "Amazon"  ]
def get_company_data(company_name,p):
    # Set up NewsAPI client
    newsapi = NewsApiClient(api_key=news_key)

    # Create empty arrays to receive responses
    headline = []
    date_published = []
    description = []

    #Search the NewsAPI for articles about a company on stock days
    #Loop through X number of pages, up to 50
    for x in range(1,p):
        for date in dates:
            amazon_articles = newsapi.get_everything(q=f"{company_name}",
                                              from_param= date,
                                              to= date,
                                              language='en',
                                              sort_by='relevancy', page=x)

        #Output headline names, date, and description
        for i in range(len(amazon_articles["articles"])):
            headline.append(amazon_articles["articles"][i]["title"])
            date_published.append(amazon_articles["articles"][i]["publishedAt"])
            description.append(amazon_articles["articles"][i]["description"])                              
    #Find the company name of each headline
    #Put the other information into an array for main Datafame
    company = []
    company_headline = []
    company_date = []
    company_sentiment = []
    #Loop through X number of  headlines. change to len(headline) when ready
    for i in range(10):
        request = requests.post(f"http://api.intellexer.com/recognizeNeText?apikey={int_key}&loadNamedEntities=true&loadRelationsTree=False&loadSentences=true", data= f"{headline[i] + description[i]}".encode('utf-8')).json()
        for y in range(len(request["entities"])):
            try:
                if request['entities'][y]["type"]== 2 and request['entities'][y]["text"] == f"{company_name}":
                    company.append(request["entities"][y]["text"])
                    company_headline.append(headline[i])
                    company_date.append(date_published[i][:10])

                    #Request the sentiment
                    url = f"http://api.intellexer.com/analyzeSentiments?apikey={int_key}"
                    sentiment = requests.post(url, data= json.dumps([{"id": "1", "text": headline[i] + description[i]}]),headers = {"Content-Type": "application/json", "Cache-Control": "no-cache"}).json()
                    company_sentiment.append(float(sentiment['sentiments'][0]['w']))

                    break
                else: 
                    continue
            except:
                continue
    company_cols = {"Headline": company_headline,
                "Company": company,
                "Date": company_date,
                "Sentiment": company_sentiment}
    company_data = pd.DataFrame(company_cols)
    #export to csv when ready
    company_data.to_csv(f"{company_name}_sentiment.csv")

In [13]:
#run this function with company name and number of pages
get_company_data(companies[0],10)

In [ ]:
#output all company csv's, get all pages
#for company_name in companies:
#    get_company_data(company_name,50)

In [ ]:
def get_stock_csv(ticker):
    change = []
    stock_date = []
    date = []
    stock = requests.get(f"https://api.iextrading.com/1.0/stock/{ticker}/chart/1m")
    stocks = stock.json()
    
    for i in range(len(stocks)):
        change.append(stocks[i]['changePercent'])
        stock_date.append(stocks[i]['date'])
    stocks_df = pd.DataFrame({"Date":stock_date, "Percent Change": change})
    #print to csv when ready
    #stocks_df.to_csv(f"{ticker}.csv")
    

In [ ]:
#Print out CSV's for each company 

for ticker in Tickers:
    get_stock_csv(ticker)